In [19]:
#Importing the packags
# pip install --user -U nltk
# pip install -U scikit-learn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
import pickle
import nltk
nltk.download('stopwords')
%matplotlib inline 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
df = pd.read_csv('spam.csv', encoding = 'latin1',nrows = 1613)
df = df.sample(frac = 1)
df

,Category,Message
478,ham,Tension ah?what machi?any problem?
497,ham,Nope... Think i will go for it on monday... So...
1167,ham,I am not having her number sir
1287,ham,"Night has ended for another day, morning has c..."
1442,ham,Ya:)going for restaurant..
...,...,...
594,ham,You still at grand prix?
674,ham,Ditto. And you won't have to worry about me sa...
553,ham,"Sure, if I get an acknowledgement from you tha..."
28,ham,"I'm back &amp; we're packing the car now, I'll..."


In [21]:
data= df.sort_index()

In [22]:
data

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
1608,ham,Jus telling u dat i'll b leaving 4 shanghai on...
1609,ham,Are your freezing ? Are you home yet ? Will yo...
1610,ham,You all ready for * big day tomorrow?
1611,ham,I'll probably be around mu a lot


In [23]:
normal= data.where(pd.notnull(data)) # converting mail data to normal

In [24]:
normal.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1613 entries, 0 to 1612
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  1613 non-null   object
 1   Message   1613 non-null   object
dtypes: object(2)
memory usage: 37.8+ KB


In [25]:
normal.shape

(1613, 2)

In [26]:
normal.loc[data['Category']== 'spam', 'Category']= 0
normal.loc[data['Category']== 'ham', 'Category']= 1


In [27]:
x= normal['Message']
y= normal['Category']
print(x)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
1608    Jus telling u dat i'll b leaving 4 shanghai on...
1609    Are your freezing ? Are you home yet ? Will yo...
1610                You all ready for * big day tomorrow?
1611                     I'll probably be around mu a lot
1612                                                  645
Name: Message, Length: 1613, dtype: object


In [28]:
print(y)

0       1
1       1
2       0
3       1
4       1
       ..
1608    1
1609    1
1610    1
1611    1
1612    1
Name: Category, Length: 1613, dtype: object


In [29]:
# test and train data
x_train, x_test, y_train, y_test= train_test_split(x,y, test_size=0.2, random_state = 3)

In [30]:
print(x.shape)
print(x_train.shape) #80% 
print(x_test.shape) #29%

(1613,)
(1290,)
(323,)


In [31]:
print(y.shape)
print(y_train.shape)
print(y_test.shape)

(1613,)
(1290,)
(323,)


In [32]:
#transform the text data to feature vector that can be used as input in logistic regression
feature_extraction = TfidfVectorizer(min_df= 1, stop_words='english', lowercase= True)
x_train_feature= feature_extraction.fit_transform(x_train)
x_test_feature= feature_extraction.transform(x_test)
y_train= y_train.astype('int')
y_test= y_test.astype('int')

In [33]:
print(x_train_feature)

  (0, 3053)	0.7071067811865476
  (0, 2493)	0.7071067811865476
  (1, 1141)	0.267404216783935
  (1, 2763)	0.2803538264705278
  (1, 1944)	0.23612482615929045
  (1, 1621)	0.891146331025802
  (2, 1944)	0.1926052792059469
  (2, 1180)	0.5458195115602718
  (2, 1002)	0.30160228360401453
  (2, 2854)	0.32079570334176793
  (2, 1924)	0.30160228360401453
  (2, 388)	0.26879091584267734
  (2, 1294)	0.3536070711031052
  (2, 2574)	0.2943052840802101
  (2, 2735)	0.310232802227712
  (3, 3460)	1.0
  (4, 1718)	0.450752031435408
  (4, 1001)	0.3746398397889329
  (4, 2814)	0.3908177658832344
  (4, 433)	0.4913242227674146
  (4, 2519)	0.5121811970307082
  (5, 1559)	0.5210433890036082
  (5, 527)	0.5846472730954992
  (5, 2124)	0.6218531601894705
  (6, 1142)	0.6192056376212609
  :	:
  (1285, 282)	0.2805222130603649
  (1285, 2479)	0.2805222130603649
  (1285, 64)	0.2805222130603649
  (1286, 1831)	0.25680130209668084
  (1286, 1140)	0.3338138961948207
  (1286, 2222)	0.5236479014085331
  (1286, 1270)	0.5236479014085331


In [34]:
model= LogisticRegression()

In [35]:
model.fit(x_train_feature,y_train)

LogisticRegression()

In [36]:
prediction_on_traning_Data= model.predict(x_train_feature)
accuracy_on_traning_Data= accuracy_score(y_train,prediction_on_traning_Data )

In [37]:
prediction_on_traning_Data

array([1, 1, 1, ..., 1, 1, 1])

In [38]:
accuracy_on_traning_Data

0.9209302325581395

In [39]:
prediction_on_test_Data= model.predict(x_test_feature)
accuracy_on_test_Data= accuracy_score(y_test,prediction_on_test_Data )

In [40]:
prediction_on_test_Data

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [41]:
accuracy_on_test_Data

0.9256965944272446

In [44]:
input_your_mail =input("give your email:")
input_mail=[str(input_your_mail)]
input_data_features= feature_extraction.transform(input_mail)
prediction= model.predict(input_data_features)
print(prediction)
if (prediction[0]==1):
    print('ham mail')
else:
    print('spam mail')

[0]
spam mail
